# Bubble Oscillations -- Brightfield
### Brice Saint-Michel, TU Delft
(can be reached going for bsaintmichel on gmail)

* This program goes and fetches videos to process bubble oscillation data
* Some formatting is needed to get the correct metadata for the experiments 
(i.e. include frequency, number of cycles, voltage and gain)
* Other than that most of the processing is automatic
* As the PFV software has changed during experiments I will try to keep compability with 
both versions of the .cih (or .cihx) files

In [1]:
main_folder = 'G:\\Data\\Bubble Oscillation\\20210125\\'
exp_folder  = 'Bubble'
filetypes = ('.avi', '.mp4')
show_pics = False # Displays live data processing (for monitoring)
reprocess = False # Does nothing if a saved data file is already present

### Here, we define the functions that will look into the metadata (string file + .cih file) of the experiment

In [17]:
import glob, re
vidfiles = (glob.glob(main_folder + '/**/*.avi', recursive=True) + 
            glob.glob(main_folder + '/**/*.mp4', recursive=True))

def parsevid(filepath):
    splits = filepath.rsplit('\\', 1)
    filename = splits[1]
    info = {'folder':splits[0], 'name':splits[1], 'ncyc':100, 'volt':0,
            'gain':0, 'numexp':0, 'freq':1}
        # I am using default values here, they will be overwritten
        # by the program

    freqmatch = re.findall('(\d*p?\d*?)kHz', filename)
    voltmatch = re.findall('(\d*)mV', filename)
    gainmatch = re.findall('(\d*)pc', filename)
    cycmatch = re.findall('(\d*)cyc', filename)
    numexpmatch = re.findall('_(\d*)[\W]', filename)
    
    if voltmatch:
        info['volt'] = int(voltmatch[0])
    if gainmatch: 
        info['gain'] = int(gainmatch[0])
    if cycmatch:
        info['cyc'] = int(cycmatch[0])
    if freqmatch:
        freqstr = freqmatch[0].replace('p', '.')
        info['freq'] = int(freqstr)*1000
    if numexpmatch:
        info['numexp'] = int(numexpmatch[0])

    # Handling the cih(X) file
    cihmatch = glob.glob(info['folder'] + '\\' + info['name'].split('.')[0] + '.cih*')
    if not cihmatch:
        info['skip'] = True # There should always be a CIH(X) accompanying file with the video file
    if cihmatch[0].endswith('.cih'):
        info = readcih(cihmatch[0], info)
    elif cihmatch[0].endswith('.cihx'):
        info = readcihX(cihmatch[0], info)

    return info

# Reads Metadata file from (old) PFV version
# Data usually does NOT contain any info on **magnification** (and hence pixel size) 
def readcih(cihfilename, info):
    cihfile = open(cihfilename)
    cihdata = cihfile.read()
    cihfile.close()

    datematch = re.findall('\nDate : (\d+/\d+/\d+)', cihdata)
    timematch = re.findall('\nTime : (\d+:\d+)', cihdata)
    fpsmatch  = re.findall('\nRecord Rate\(fps\) : (\d+)', cihdata)
    shuttermatch = re.findall('Shutter Spped\(s\) : 1/(\d+)', cihdata)
    frame0match = re.findall('Start Frame : (\d+)', cihdata)
    nframesmatch = re.findall('\nTotal Frame : (\d+)', cihdata)
    decimmatch = re.findall('Save Step : (\d+)', cihdata)
    vidheightmatch = re.findall('Image Height : (\d+)', cihdata)
    vidwidthmatch = re.findall('Image Width : (\d+)', cihdata)
    
    if datematch: info['date'] = datematch[0]
    if timematch: info['time'] = timematch[0]
    if fpsmatch: info['fps'] = int(fpsmatch[0])
    if shuttermatch: info['shutter'] = int(shuttermatch[0])
    if frame0match: info['frame0'] = int(frame0match[0])
    if nframesmatch: info['nframes'] = int(nframesmatch[0])-1 # It simplifies things later on due to me skipping first frame
    if decimmatch: info['decim'] = int(decimmatch[0])
    if vidwidthmatch: info['width'] = int(vidwidthmatch[0])
    if vidheightmatch: info['height'] = int(vidheightmatch[0])
    info['fps'] = info['fps']/info['decim'] # Account for decimation
    
    return info

# Reads Metadata file from (old) PFV version
# In that case I have usually specified metadata 
def readcihX(cihfilename, info):
    cihfile = open(cihfilename)
    cihdata = cihfile.read()
    cihfile.close()

    datematch = re.findall('<date>(\d+)</date>', cihdata)
    timematch = re.findall('<time>(\d+)</time>', cihdata)
    fpsmatch  = re.findall('<recordRate>(\d+)</recordRate>', cihdata)
    shuttermatch = re.findall('<shutterSpeed>(\d+)</shutterSpeed>', cihdata)
    frame0match = re.findall('<startFrame>(\d+)</startFrame>', cihdata)
    nframesmatch = re.findall('<totalFrame>(\d+)</totalFrame>', cihdata)
    decimmatch = re.findall('<skipFrame>(\d+)</skipFrame>', cihdata)
    vidheightmatch = re.findall('<height>(\d+)</height>', cihdata)
    vidwidthmatch = re.findall('<width>(\d+)</width>', cihdata)
    magnificationmatch = re.findall('<magnification>(\d+)</magnification>', cihdata)

    if datematch: info['date'] = datematch[0]
    if timematch: info['time'] = timematch[0]
    if fpsmatch: info['fps'] = int(fpsmatch[0])
    if shuttermatch: info['shutter'] = int(shuttermatch[0])
    if frame0match: info['frame0'] = int(frame0match[0])
    if nframesmatch: info['nframes'] = int(nframesmatch[0])-1
    if decimmatch: info['decim'] = int(decimmatch[0])
    if vidwidthmatch: info['width'] = int(vidwidthmatch[0])
    if vidheightmatch: info['height'] = int(vidheightmatch[0])
    if magnificationmatch: info['pixsize'] = 2e-5/float(magnificationmatch[0])
    info['fps'] = info['fps']/info['decim'] # Account for decimation

    print(info)

    return info

## The mighty routines used to process the files :

Libraries :

* `Pyav` : `>> pip install av`
* `scikit-image` : `>> pip install skimage`
* `scipy` : `>> pip install scipy`

In [3]:
import numpy as np
import numpy.matlib as matlib
import av

import skimage.filters as skfilt
import skimage.measure as skmeas
import skimage.morphology as skimmorph

import scipy.interpolate as spint
import scipy.special as spec
import scipy.fft as fft

# This routine extracts a line from a 2d map and also recasts the picture
# surroundings of the line into a (r, theta) map to check for accuracy
def image_padding(myimage, padsize=12):
    mshape = np.shape(myimage)
    myimage2 = np.zeros((mshape[0] + 2*padsize, mshape[1] + 2*padsize))
    myimage2[padsize:-padsize, padsize:-padsize] = myimage
    myimage2[0:padsize, padsize:-padsize]  = matlib.repmat(myimage[0,:], padsize, 1)
    myimage2[padsize+mshape[0]:, padsize:-padsize] = matlib.repmat(myimage[0,-1], padsize, 1)
    myimage2[padsize:-padsize, 0:padsize]  = matlib.repmat(np.transpose([myimage[:,0]]), 1, padsize)
    myimage2[padsize:-padsize, padsize+mshape[0]:] = matlib.repmat(np.transpose([myimage[:,-1]]), 1, padsize)
    myimage2[0:padsize, 0:padsize] = myimage[0,0]*np.ones((padsize, padsize))
    myimage2[-padsize:, 0:padsize] = myimage[-1,0]*np.ones((padsize, padsize))
    myimage2[0:padsize, -padsize:] = myimage[0,-1]*np.ones((padsize, padsize))
    myimage2[-padsize:, -padsize:] = myimage[-1,-1]*np.ones((padsize, padsize))
    
    return myimage2

# This routine extracts a line from a 2d map and also recasts the picture
# surroundings of the line into a (r, theta) map to check for accuracy
def edge_detection(myimg, thrs, doCDS=False):

    # Image filtering, filling the "bright hole" at the centre
    myimg = skfilt.gaussian(myimg, sigma=0.75, mode='reflect', preserve_range=True)
    seed = np.copy(myimg)
    seed[1:-1, 1:-1] = myimg.min()
    filled = skimmorph.reconstruction(seed, myimg, method='dilation')
    contour = skmeas.find_contours(filled, thrs)    
    if np.shape(contour)[0] == 2:
        print('Two objects found // Use TwoBubblesOsc for two Bubbles !')
    contour = contour[0]
        # My function works with only one contour !
        # If the program crashes here, normally it means that something is wrong with your picture
        # - check range [0-1 vs 0-255]
        # - check if filtering did not mess up everything

    # Check contour in normalised form, extract mean radius, etc
    xctr, yctr = np.mean(contour[:,0]), np.mean(contour[:,1])
    xlocal, ylocal = contour[:,0] - xctr, contour[:,1] - yctr
    radius, angle = np.sqrt(xlocal**2 + ylocal**2), np.arctan2(ylocal, xlocal)
    angle_i = np.linspace(-np.pi, np.pi, 360, endpoint=False)
    interpol1d = spint.interp1d(angle, radius, kind='linear', fill_value='extrapolate')
    radius_i = interpol1d(angle_i)
    x_i, y_i  = xctr + radius_i*np.cos(angle_i), yctr + radius_i*np.sin(angle_i) # 0.5 is a display thing 
    avgrad = np.mean(radius)

    # Build local map around the contour; padding the image usually helps
    padsize = 12 
    padimg = image_padding(myimg, padsize)
    rscan = np.linspace(0.7,1.3,30)*avgrad
    x_polar = xctr + padsize + np.ravel(np.dot(np.transpose([rscan]), [np.cos(angle_i)]))
    y_polar = yctr + padsize + np.ravel(np.dot(np.transpose([rscan]), [np.sin(angle_i)]))
    xpad, ypad = np.arange(np.shape(padimg)[0]), np.arange(np.shape(padimg)[1])
    interpol2d = spint.RectBivariateSpline(xpad, ypad, padimg)
    local_interface = interpol2d.ev(x_polar, y_polar)
    local_interface = np.reshape(local_interface, (30,360))

    return{'image': filled, 'ROI': local_interface, 'theta':angle_i, 'interface':radius_i/avgrad, 
           'xctr':xctr, 'yctr':yctr, 'contourx':x_i, 'contoury':y_i, 'rad':avgrad}

### This routine tries to determine the orientation of a bubble.
### Note : for some shape modes (especially even ones), multiple
### solutions are possible.
def compute_orientation(bubble, info):
    indices = np.arange(0,360)
    score = np.zeros((info['nframes'],180))

    for frame in range(info['nframes']):
        signal = bubble['interface'][frame]
        for shift in range(180):
            shifted_indices = np.roll(indices, shift)
            score[frame,shift] = np.sum((signal[shifted_indices[179::-1]] 
                                        - signal[shifted_indices[180:]])**2)
        print('>> %s : Orientation -- Frame %d out of %d' % (info['name'], frame, info['nframes']), end='\r')

    score = skfilt.gaussian(score, sigma=2) # We filter a bit to avoid jittering
    bubble['orientation'] = np.argmin(score, axis=1)
    return 0

### This routine works out a Legendre analysis of the oscillations 
def legendre_analysis(bubble, info):
    bubble['projtype'] = 'legendre'
    thetas = np.arange(360)
    thetascale = np.linspace(0,np.pi,180, endpoint=False)
    costhetascale = np.cos(thetascale)
    
    bubble['approx'] = np.zeros((info['nframes'],np.size(thetascale)))
    projeast = np.zeros((info['nframes'],15))
    projwest = np.zeros((info['nframes'],15))
    bubble['modes'] = np.zeros((info['nframes'],15))

    for frame in range (info['nframes']):
        
        oriented_signal = np.roll(bubble['interface'][frame], bubble['orientation'][frame])
        eastsignal = oriented_signal[:180]
        westsignal = oriented_signal[179::-1]

        # # Synthetic signal [for tests]
        # mode = spec.legendre(3)
        # r_simul = 1 + 0.2*np.cos(frame/100)*mode(costhetascale)
        # eastsignal = r_simul[:]
        # westsignal = r_simul[:]

        for mode in range(15):
            polynow = spec.legendre(mode)
            
            projeast[frame,mode] = -(2*mode+1)/2*np.trapz(eastsignal*polynow(costhetascale), x=costhetascale)
            projwest[frame,mode] = -(2*mode+1)/2*np.trapz(westsignal*polynow(costhetascale), x=costhetascale)
            bubble['modes'][frame,mode] = 0.5*(projeast[frame,mode] + projwest[frame,mode])
            bubble['approx'][frame] += bubble['modes'][frame,mode]*polynow(costhetascale)

        print('>> %s : Legendre Analysis -- Frame %d out of %d' % (info['name'], frame, info['nframes']), end='\r')
        
    # # Debugging with the synthetic signal
    # p1 = figure()
    # p1.line(x=costhetascale*(1+r_simul), y=np.sin(thetascale)*(1+r_simul), line_color='red', line_dash='dashed')
    # p1.line(x=costhetascale*(1+r_simul), y=-np.sin(thetascale)*(1+r_simul), line_color='red', line_dash='dashed')
    # p1.line(x=costhetascale*(1+approx), y=np.sin(thetascale)*(1+approx), line_color='black')
    # p1.line(x=costhetascale*(1+approx), y=-np.sin(thetascale)*(1+approx), line_color='black')
    # show(p1)
    return 0
    
### This routine works out a Fourier analysis of the oscillations
### It is a bit broken because oscillation amplitudes cannot be negative stricto sensu . . .
def fourier_analysis(bubble, info):
    bubble['projtype'] = 'fourier'
    bubble['modes'] = np.zeros((info['nframes'],15))
    bubble['approx'] = np.zeros((info['nframes'],360))
    theta_integration = np.arange(361)*np.pi/180   # We need to "loop back" to the initial point
    theta_projection = np.arange(360)*np.pi/180
    projector = lambda theta, mode : np.cos(theta*mode) + 1j*np.sin(theta*mode)
    
    for frame in range(info['nframes']):
        oriented_signal = np.roll(bubble['interface'][frame], bubble['orientation'][frame])
        oriented_signal = np.append(oriented_signal, oriented_signal[0])
        bubble['modes'][frame,0] = np.mean(oriented_signal)
        bubble['approx'][frame,:] = bubble['modes'][frame,0]
        for mode in range(1,15):
            complex_mode = np.trapz(oriented_signal*projector(theta_integration, mode), x=theta_integration)/np.pi
            bubble['modes'][frame,mode] = np.real(complex_mode) # I KNOW IT SHOULD BE ABS
            bubble['approx'][frame] += np.real(complex_mode*np.conj(projector(theta_projection, mode)))
             # The np.real is needed to convert from (type) complex to real even if we expect imag to be 0

        print('>> %s : Fourier Contour -- Frame %d out of %d' % (info['name'], frame, info['nframes']), end='\r')
        bubble['approx'][frame] = np.roll(bubble['approx'][frame], -bubble['orientation'][frame])


### Work out the spectrograms for the shape modes
def compute_spectrograms(bubble,info):
    DT = int(np.round(4*info['fps']/info['freq']))
    number_of_windows = int(np.floor(info['nframes']/DT))
    f_scale_spectro = fft.fftfreq(DT, d=info['freq']/info['fps'])
    half_freq_idx = np.argmin(np.abs(f_scale_spectro - 0.5))
    bubble['spectrogram_modes'] = np.zeros((number_of_windows, 15))  # 15 is the number of modes (from rawmodes)
    bubble['spectrogram_frames'] = np.zeros(number_of_windows)

    for wno in range(10):
        indices = np.arange(wno*DT,(wno+1)*DT)
        rawmodes = bubble['modes'][indices,:]
        formattedmodes = rawmodes - np.dot( np.ones((np.size(indices), 1)) , [np.mean(rawmodes, axis=0)] )
        bubble['spectrogram_modes'][wno,:] = 2/DT*np.abs(fft.fft(formattedmodes, axis=0))[half_freq_idx,:]
        bubble['spectrogram_frames'][wno] = np.mean(indices)

### A simple tool to Fourier transform the time series of the bubble radius
def fourier_rad_timeseries(bubble, info):
    frames_per_period = float(info['fps']/info['freq'])
    freqlist = fft.fftfreq(np.size(bubble['rad']), d=1/frames_per_period)
    radfft = np.abs(fft.fft(radius - np.mean(radius)))
    posfreq = freqlist > 0
    bubble['time_fourier_freqs'] = freqlist[posfreq]
    bubble['time_fourier_rad'] = radfft[posfreq]



### Data processing for one video
def process_one_video(vidfile):  
    print('This is Process_one_video ...')  
    info = parsevid(vidfile)
    container = av.open(info['folder'] + '\\' + info['name'])
    contents = container.decode(video=0)
    refimg = next(contents).to_ndarray(format='gray')
    refimg = next(contents).to_ndarray(format='gray') # Skipping the (sometimes messed up) first frame
    thrs = skfilt.threshold_otsu(refimg)
    info['threshold'] = thrs

    edge_info = edge_detection(refimg, thrs)
    bubble = dict(x=np.zeros(info['nframes']), y=np.zeros(info['nframes']),  rad=np.zeros(info['nframes']), 
                  interface=np.zeros((info['nframes'], 360)), theta=edge_info['theta'], idx=np.arange(info['nframes'])) 
    bubble['x'][0] = edge_info['xctr']
    bubble['y'][0] = edge_info['yctr']
    bubble['rad'][0] = edge_info['rad']
    bubble['interface'][0] = edge_info['interface']
    
    # Note : due to the 'anti-jittering filter' for bubble orientation
    # we have to sequentially do all contour detections, then all orientations,
    # then finally the legendre/fourier projection

    for frameno in range(1, info['nframes']):
        nowimg = next(contents).to_ndarray(format='gray')
        edge_info = edge_detection(nowimg, thrs)
        bubble['x'][frameno] = edge_info['xctr']
        bubble['y'][frameno] = edge_info['yctr']
        bubble['rad'][frameno] = edge_info['rad']
        bubble['interface'][frameno] = edge_info['interface']
        print('>> %s : Contour -- Frame %d out of %d' % (info['name'], frameno, info['nframes']), end='\r')

    container.close()  
    compute_orientation(bubble,info)

    # # Choose your team (wisely) for shape oscillation detection method
    legendre_analysis(bubble,info)
    fourier_analysis(bubble,info)

    fourier_rad_timeseries(bubble, info)
    compute_spectrograms(bubble,info)

    return bubble, info   

## Defining here the routine used to plot a snapshot of the data 

Library used : 

* `bokeh` : `>> pip install bokeh`

In [113]:
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import LinearColorMapper, ColumnDataSource
from bokeh.palettes import grey

### This routine allows "snapshots" to be shown --------------------------
def plot_snapshot(info, frameno=0):
    output_notebook()

    # Going to the right frame 
    container = av.open(info['folder'] + '\\' + info['name'])
    contents = container.decode(video=0)
    for fno in range(frameno):
        frame = next(contents)
    frame = next(contents).to_ndarray(format='gray')
    container.close()
    
    # ColumnDataSources used for display
    edge_info = edge_detection(frame, info['threshold'])
    CDS_img  = ColumnDataSource({'image':[edge_info['image']], 'ROI':[edge_info['ROI']]})
    CDS_contour = ColumnDataSource( {'contourx':edge_info['contourx']+0.5, 'contoury':edge_info['contoury']-0.5,
                                         'interface':edge_info['interface'], 'theta':edge_info['theta']})
    CDS_xy = ColumnDataSource({'x':[edge_info['xctr']], 'y':[edge_info['yctr']], 'idx':[frameno]})

    # Generating glyphs for the figure
    vmapper = LinearColorMapper(palette=grey(256), low=0, high=255)
    p1 = figure(width=400, height=400, title='Image : ' + str(frameno) + ' / ' + str(info['nframes']-1))
    p2 = figure(width=500, height=250, title='Contour : ' + info['name'])
    p1.image('image', x=-0.5, y=-0.5, dw=128-0.5, dh=128+0.5, color_mapper=vmapper, source=CDS_img)
    p2.image('ROI', x=-np.pi, dw=2*np.pi, y=0.7, dh=0.6, color_mapper=vmapper, source=CDS_img)
    p1.line(x='contourx', y='contoury', line_width=2, line_dash='dotted', line_color='lime', source=CDS_contour)
    p2.dot(x='theta', y='interface', line_color='lime', size=5, source=CDS_contour)
    p1.cross(x='x', y='y', size=20, line_color='lime', source=CDS_xy)
    p2.line(np.pi*np.array([-1.,1.]), 0.99*np.array([1.,1.]), line_color='white', line_dash='dotted')
    p2.line(np.pi*np.array([-1.,1.]), 1.01*np.array([1.,1.]), line_color='white', line_dash='dotted')
    show(row(p1,p2))

plot_snapshot(info, 512)

Loading BokehJS ...

### The actual data processing happens here :